# improts

In [8]:
import pandas as pd
from notion_client import Client


# function definition

In [9]:
def get_text_content(prop_value, prop_type='rich_text'):
    if not prop_value:
        return ''
    try:
        if prop_type == 'date':
            return prop_value.get('date', {}).get('start', '')
        if prop_type == 'rich_text':
            texts = prop_value.get('rich_text', [])
            if not texts:
                return ''
            first_text = texts[0].get('text', {})
            return first_text.get('content', '')
        elif prop_type == 'title':
            titles = prop_value.get('title', [])
            if not titles:
                return ''
            first_title = titles[0].get('text', {})
            return first_title.get('content', '')
        elif prop_type == 'email':
            return prop_value.get('email', '')
    except (AttributeError, IndexError, KeyError):
        return ''
    return ''

# notions setup

In [10]:
# Initialize Notion client
NOTION_TOKEN = "ntn_S6159294934albrajfceBHL4szrrrMllKAcFNUGM62v7JI"
DATABASE_ID = "18316f1f61d680a2921bd08b8c62f895"
notion = Client(auth=NOTION_TOKEN)

# Pull database

In [12]:
# create empty data arrey
data = []

# call notion API 
response = notion.databases.query(database_id=DATABASE_ID)

# iterate through each page and collect data
for page in response['results']:
    # get page properties
    properties = page.get('properties', {})
    # select the fields we want to extract
    row = {
        'customer name': get_text_content(properties.get('Contact 1 name')),
        'Phone': properties.get('Contact 1 phone', {}).get('phone_number', ''),
        'Email': get_text_content(properties.get('Contact 1 Email address'), 'email'),
        'home address': get_text_content(properties.get('Address')),
        'Next Follow Up Date': properties.get('Next Follow Up Date', {}).get('date', {}).get('start', '') if properties.get('Next Follow Up Date') and properties.get('Next Follow Up Date').get('date') else '',
        'Claim': properties.get('Status of Claim', {}).get('status', {}).get('name', ''),
        'Lead': properties.get('Status of lead', {}).get('status', {}).get('name', ''),   
        'call id' : get_text_content(properties.get('Last call ID')),
        'First_call_transcription': get_text_content(properties.get('cold call transcription'))
    }
    # append data to the data arrey
    data.append(row)
    
# convert data to a dataframe   
df = pd.DataFrame(data)
df

,customer name,Phone,Email,home address,Next Follow Up Date,Claim,Lead,call id,First_call_transcription
0,test,131243,sda@askld.com,test1,,Not approved by client,New,123aa,
1,test,131243,sda@askld.com,test1,2025-02-06T00:00:00.000+02:00,Not approved by client,Did not answered,,
2,test,131243,sda@askld.com,asd,2025-01-25T00:00:00.000+02:00,Done,Fund recived,,
3,test,131243,sda@askld.com,test1,2025-01-25T00:00:00.000+02:00,Sent to Attorney,Client singed documents,,
4,test,131243,sda@askld.com,test1,2025-01-25T00:00:00.000+02:00,Sent to Attorney,Interested,,\nuser: Hello? \n assistant: Hi\nis this John?...
5,test,131243,sda@askld.com,test1,2025-01-25T00:00:00.000+02:00,Approved and waiting for filled documents,Documents sent for signiture,1235aa,as’;ldnkj. agskff hoaisf;lioo jalos;fj lkashf....
6,test,131243,sda@askld.com,test1,2025-01-25T00:00:00.000+02:00,Not approved by client,Asked for follow up,,


# Pull database only for cold call

In [ ]:
cold_call_df = df[df['Lead'].isin(['New', 'Did not answered'])]
cold_call_df

# Pull database only for closer

In [ ]:
closer_df = df[df['Lead'].isin(['Interested'])]
closer_df

# update database test

In [5]:
# Find the row with call id "123aa"
mask = df['call id'] == "123aa"

# Update the "First_call_transcription" column for the matched row
df.loc[mask, 'First_call_transcription'] = """
user: Hello? \n assistant: Hi
is this John? \n user: Yes. It is. \n assistant: Hi John
my name is Cristin and I'm calling from TrueClaim Services. I've spent the last several days trying to find you. We specialize in auditing public records to search for unclaimed funds held by government agencies and owed to individuals. I believe I've found some funds that might belong to you. Can you confirm if you used to live at 123 Main St before? \n user: Yes. It is. \n assistant: John
it appears that the local courts are holding onto some funds that belong to you
likely from a past foreclosure. These funds are at risk of being turned over to the government if not claimed in a timely manner. We'd like to help you recover them. Our service takes care of everything
from preparing and submitting the necessary paperwork to dealing with the bureaucracy
all at no upfront cost or risk to you. We only get paid if you receive your money back. Would you like to know more about the process and potentially recovering your funds? \n user: Yeah. Of course. \n assistant: I'm going to send over some information documents for you to review at your convenience. Additionally
I'm setting up a call with one of our senior representatives who will guide you through the process and answer any questions you may have. When would you prefer us to reach out to you to schedule this call? \n user: It's today. \n user: When \n assistant: I can arrange for the senior representative to call you today. Would you be available to take the call within the next hour or would you prefer a specific time that suits you better? \n user: Yeah. Next hour. \n assistant: To ensure you receive all necessary documents and updates on your request
could you please share your email address with me? \n user: Yeah. Sure. And email is john@gmail.com. \n assistant: Just to confirm
your email is john@gmail.com
correct? \n user: Yes. It is. \n assistant: Thank you so much for taking the time to chat with me today
John. \n 
"""

# Update the Notion database 
for index, row in df[mask].iterrows():
    page_id = response['results'][index]['id']
    notion.pages.update(
        page_id=page_id,
        properties={
            'cold call transcription': {
                'rich_text': [{'text': {'content': row['First_call_transcription']}}]
            }
        }
    )

print("Notion database updated successfully.")


Notion database updated successfully.
